In [ ]:
import earthaccess
import xarray as xr
import h5py

import cartopy.crs as ccrs

import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import cmocean
import numpy as np

auth = earthaccess.login()

results = earthaccess.search_data(
    short_name = "ATL10",
    # granule_name="", # if we know the file name we can use it here to donwload or open it directly e.g. ATL10-02_20201231232719_01221001_006_01.h5
    cloud_hosted= True,
    temporal=("2021-01","2021-02"),
    count = 1
)
results[0]

## Open ATL data with earthaccess' `.open()`

In [ ]:
files = earthaccess.open(results, "ATL10")

### Reading ATL10 data from a single file

Because ATL10 is not a gridded prduct we need to extrac coordinates and variables from their groups inside the HDF5 file.

In [ ]:
## Based on the READ function form Younghyun Koo for the sea ice tutorial at the IS2 hackweek
def read_atl10(filename):

    # Create a list for saving ATL10 beam track data
    tracks = []

    with h5py.File(filename,'r') as f:

        # Check the orbit orientation
        orient = f['orbit_info/sc_orient'][0]

        if orient == 0:
            strong_beams = [f"gt{i}l" for i in [1, 2, 3]]
        elif orient == 1:
            strong_beams = [f"gt{i}r" for i in [1, 2, 3]]
        else:
            strong_beams = []

        for beam in strong_beams:

            lat = f[beam]['freeboard_segment/latitude'][:]
            lon = f[beam]['freeboard_segment/longitude'][:]
            seg_x = f[beam]['freeboard_segment/seg_dist_x'][:] / 1000 # (m to km)
            seg_len = f[beam]['freeboard_segment/heights/height_segment_length_seg'][:]
            fb = f[beam]['freeboard_segment/beam_fb_height'][:]
            surface_type = f[beam]['freeboard_segment/heights/height_segment_type'][:]
            fb[fb>100] = np.nan

            df = pd.DataFrame({'lat': lat, 'lon': lon, 'seg_x': seg_x, 'seg_len': seg_len,
                              'freeboard': fb, 'stype': surface_type})
            df['beam'] = beam
            df = df.dropna().reset_index(drop = True)
            gdf = gpd.GeoDataFrame(
                    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
            )
            tracks.append(gdf)
        return tracks

In [ ]:
def plot_freeboard(tracks, granule):
    # test extent
    if tracks[0].lat.max() < 0:
        target_projection = ccrs.SouthPolarStereo()
        target_crs = "EPSG:3031"
        target_extent = [-180, 180, -90, -60]
    else:
        target_projection = ccrs.NorthPolarStereo()
        target_crs = "EPSG:3413"
        target_extent = [-180, 180, 90, 60]
    
    axd = plt.figure(layout="constrained", figsize=(12, 5)).subplot_mosaic(
        """
        AABBB
        AACCC
        AADDD
        """,
        per_subplot_kw={
            "A": {"projection": target_projection},
            ("B", "C", "D"): {"xscale": "log"}
        },
    )
    cmap = cmocean.cm.ice
    norm = plt.Normalize(vmin=1, vmax=6)
    track_dict = ["B", "C", "D"]

    for count, track in enumerate(tracks):
        color=cmap(norm(count))
        
        location_data = gpd.GeoDataFrame(track, geometry=track.geometry.to_crs(target_crs))
        axd["A"].set_extent(target_extent, crs=ccrs.PlateCarree())

        location_data.plot(ax=axd["A"], column="freeboard", cmap=cmap)
        axd["A"].set_title(f'ATL10 Sea Ice Freeboard \n {granule}')
        axd["A"].set_xlabel('X (m)')
        axd["A"].set_ylabel('Y (m)')
        axd["A"].coastlines()
        # we mask only sea ice freeboard
        mask_ice = (track.stype == 1)
        axd[track_dict[count]].scatter(track.seg_x[mask_ice], track.freeboard[mask_ice], s = 1, color = cmap(norm(count)), label = f"Beam {track['beam'][0]}")
        axd[track_dict[count]].axhline(0.15, color = "b", ls = "--")
        axd[track_dict[count]].set_xlabel("Along-track distance (km)")
        axd[track_dict[count]].set_ylabel("Freeboard (m)")
        axd[track_dict[count]].legend()
        
    return axd

# we only plot 1 file
file = files[0]
tracks = read_atl10(file)
# plot the tracks
granule_name = results[0]["umm"]["GranuleUR"]
plot_freeboard(tracks, granule_name)
plt.show()

We can also take advantage of Geopandas `explore()` to interactively explore the location and values of the data (although only on pseudo mercator projection)

In [ ]:
index = 0
tracks[index].explore(zoom_start=1)